# Losses subpackage

The `opr.losses` subpackage contains ready-to-use loss functions implemented in PyTorch, featuring a common interface.

## Usage example - Batch Hard Triplet Margin Loss

In [1]:
import torch

from opr.models.place_recognition import MinkLoc3D
from opr.datasets import OxfordDataset
from opr.samplers import BatchSampler
from opr.losses import BatchHardTripletMarginLoss


INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
2025-03-19 16:37:43.420 | WARNING  | opr.models.place_recognition.pointmamba:<module>:16 - The 'pointmamba' package is not installed. Please install it manually if neccessary.


### Data source

In this example, we use a pre-processed version of the Oxford RobotCar dataset.
We use the same subsample of tracks and preprocessed point clouds as described in the 
[PointNetVLAD paper](https://openaccess.thecvf.com/content_cvpr_2018/html/Uy_PointNetVLAD_Deep_Point_CVPR_2018_paper.html).
Additionally, we created the files "train.csv", "val.csv", and "test.csv."

You can download our version of the dataset via the following link:

- [Kaggle](https://www.kaggle.com/datasets/creatorofuniverses/oxfordrobotcar-iprofi-hack-23)

In [2]:
dataset = OxfordDataset(
    dataset_root="/home/docker_opr/Datasets/OpenPlaceRecognition/pnvlad_oxford_robotcar",
    subset="train",
    data_to_load=["pointcloud_lidar"],
)

sampler = BatchSampler(
    dataset=dataset,
    batch_size=32,
    positives_per_group=4
)

dataloader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_sampler=sampler,
    num_workers=4,
    pin_memory=True,
    collate_fn=dataset.collate_fn,
)


/usr/local/lib/python3.10/dist-packages/albumentations/core/validation.py:87: UserWarning: This augmenter is very slow. Try to use ``ElasticTransform`` instead, which is at least 10x faster.
  original_init(self, **validated_kwargs)
/home/docker_opr/OpenPlaceRecognition/src/opr/datasets/augmentations.py:65: UserWarning: Argument(s) 'always_apply' are not valid for transform ColorJitter
  A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, always_apply=True),
/home/docker_opr/OpenPlaceRecognition/src/opr/datasets/augmentations.py:66: UserWarning: Argument(s) 'max_width, max_height, min_width, min_height, max_holes' are not valid for transform CoarseDropout
  A.CoarseDropout(max_width=96, max_height=66, min_width=32, min_height=22, max_holes=1, p=0.5),
/home/docker_opr/OpenPlaceRecognition/src/opr/datasets/augmentations.py:180: UserWarning: Argument(s) 'max_width, max_height, min_width, min_height, max_holes' are not valid for transform CoarseDropout
  A.CoarseDropout(
/

In [3]:
sample_batch = next(iter(dataloader))
sample_batch = {k: v.to("cuda") for k, v in sample_batch.items()}
sample_batch.keys()


dict_keys(['idxs', 'utms', 'pointclouds_lidar_coords', 'pointclouds_lidar_feats'])

In [4]:
model = MinkLoc3D()
model = model.to("cuda")
model.train();


In [5]:
output = model(sample_batch)
output.keys()


dict_keys(['final_descriptor'])

In [6]:
loss_fn = BatchHardTripletMarginLoss(margin=0.2)


In [7]:
idxs = sample_batch["idxs"].cpu()
positives_mask = dataset.positives_mask[idxs][:, idxs]
negatives_mask = dataset.negatives_mask[idxs][:, idxs]

loss, stats = loss_fn(output["final_descriptor"], positives_mask, negatives_mask)


In [8]:
loss


tensor(5.7753, device='cuda:0', grad_fn=<MeanBackward0>)

In [9]:
stats


{'loss': 5.775259017944336,
 'avg_embedding_norm': 22.763263702392578,
 'num_triplets': 32,
 'num_non_zero_triplets': 32.0,
 'non_zero_rate': 1.0,
 'max_pos_pair_dist': 19.81702995300293,
 'max_neg_pair_dist': 8.448206901550293,
 'mean_pos_pair_dist': 8.219305992126465,
 'mean_neg_pair_dist': 2.654839038848877,
 'min_pos_pair_dist': 1.5070229768753052,
 'min_neg_pair_dist': 1.222188115119934}